# 1. Imports and configuration

In [29]:
import pandas as pd
import os
import re
from config import CONFIG


In [30]:
raw_data_path = CONFIG["paths"]["raw_data"]
processed_data_path = CONFIG["paths"]["processsed"]

In [31]:
model_df = pd.read_csv("./results/df_20006-1.csv", na_values=["", " "])
ground_truth = pd.read_csv("./results/gt_20006-1.csv", na_values=["", " "])

In [32]:
ground_truth.head()

,part_id_1,part_id_2,connected
0,0,1,0
1,0,2,1
2,0,3,1
3,0,4,0
4,0,5,0


In [33]:
ground_truth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2278 entries, 0 to 2277
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   part_id_1  2278 non-null   int64
 1   part_id_2  2278 non-null   int64
 2   connected  2278 non-null   int64
dtypes: int64(3)
memory usage: 53.5 KB


In [34]:
model_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   part_id        68 non-null     int64  
 1   color          68 non-null     int64  
 2   x              68 non-null     float64
 3   y              68 non-null     float64
 4   z              68 non-null     float64
 5   a              68 non-null     float64
 6   b              68 non-null     float64
 7   c              68 non-null     float64
 8   d              68 non-null     float64
 9   e              68 non-null     float64
 10  f              68 non-null     float64
 11  g              68 non-null     float64
 12  h              68 non-null     float64
 13  i              68 non-null     float64
 14  part           68 non-null     object 
 15  part_name      65 non-null     object 
 16  part_cat_id    65 non-null     float64
 17  year_from      65 non-null     float64
 18  year_to     

In [35]:
gt = ground_truth

In [37]:
def make_pair_id(a, b):
    a, b = int(a), int(b)
    return f"{min(a,b)}__{max(a,b)}"

gt["pair_id"] = gt.apply(lambda r: make_pair_id(r["part_id_1"], r["part_id_2"]), axis=1)

In [42]:
# 1) expect undirected, unique pairs
assert gt["pair_id"].nunique() == len(gt), "Duplicate pairs found."

# 2) no self-edges
assert (gt["part_id_1"] != gt["part_id_2"]).all(), "Self-edges present."

# 3) ids exist in df
valid_ids = set(model_df["part_id"])
assert set(gt["part_id_1"]).issubset(valid_ids) and set(gt["part_id_2"]).issubset(valid_ids), "Unknown part_id in GT."

# 4) labels are binary
assert set(gt["connected"].unique()) <= {0,1}, "Non-binary labels."


print("Labels are binary OK")

Labels are binary OK


In [43]:
gt.head()

,part_id_1,part_id_2,connected,pair_id
0,0,1,0,0__1
1,0,2,1,0__2
2,0,3,1,0__3
3,0,4,0,0__4
4,0,5,0,0__5
